In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 12.4MB/s 
     |████████████████████████████████| 901kB 35.5MB/s 
     |████████████████████████████████| 3.3MB 36.1MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
model = AutoModel.from_pretrained("indolem/indobert-base-uncased")

# from transformers import BertTokenizer, AutoModel
# tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-large-p1")
# model = AutoModel.from_pretrained("indobenchmark/indobert-large-p1")

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, sys

import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer
from nltk.tokenize import TweetTokenizer

from drive.MyDrive.Library.utils.forward_fn import forward_sequence_classification
from drive.MyDrive.Library.utils.metrics import document_sentiment_metrics_fn
from drive.MyDrive.Library.utils.data_utils import DocumentSentimentDataset,DocumentSentimentDataLoader

In [ ]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [ ]:
# Set random seed
set_seed(26092020)

In [ ]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = DocumentSentimentDataset.NUM_LABELS

# tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-large-p1")
# model = AutoModel.from_pretrained("indobenchmark/indobert-large-p1")

# Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
count_param(model)

124443651

In [ ]:
%ls ./drive/MyDrive/Library/smsa_doc-sentiment-prosa/

dataset_kira_labs.csv
dataset_komentar_instagram_cyberbullying.csv
dataset_tweet_sentimen_tayangan_tv.csv
dataset_tweet_sentiment_cellular_service_provider.csv
dataset_tweet_sentiment_opini_film.csv
dataset_tweet_sentiment_pilkada_DKI_2017.csv
finetuning_saham.csv
test_preprocess_masked_label.tsv
train_preprocess.tsv
train_saham.tsv
valid_preprocess.tsv
valid_saham.tsv
vocab.txt
vocab_uncased.txt


In [ ]:
# train_dataset_path = './drive/MyDrive/Library/smsa_doc-sentiment-prosa/train_preprocess.tsv'
# valid_dataset_path = './drive/MyDrive/Library/smsa_doc-sentiment-prosa/valid_preprocess.tsv'
# test_dataset_path = './drive/MyDrive/Library/smsa_doc-sentiment-prosa/test_preprocess_masked_label.tsv'

train_dataset_path = './drive/MyDrive/Library/smsa_doc-sentiment-prosa/train_saham.tsv'
valid_dataset_path = './drive/MyDrive/Library/smsa_doc-sentiment-prosa/valid_saham.tsv'

In [ ]:
# train_dataset = DocumentSentimentDataset(train_dataset_path, tokenizer, lowercase=True)
# valid_dataset = DocumentSentimentDataset(valid_dataset_path, tokenizer, lowercase=True)
# test_dataset = DocumentSentimentDataset(test_dataset_path, tokenizer, lowercase=True)

# train_loader = DocumentSentimentDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=True)  
# valid_loader = DocumentSentimentDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=False)  
# test_loader = DocumentSentimentDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=False)

train_dataset = DocumentSentimentDataset(train_dataset_path, tokenizer, lowercase=False)
valid_dataset = DocumentSentimentDataset(valid_dataset_path, tokenizer, lowercase=True)


train_loader = DocumentSentimentDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=True)  
valid_loader = DocumentSentimentDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'positive': 0, 'neutral': 1, 'negative': 2}
{0: 'positive', 1: 'neutral', 2: 'negative'}


In [ ]:
text = 'enak sekali ya'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: enak sekali ya | Label : positive (45.030%)


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=3e-6)
model = model.cuda()

In [ ]:
# Train
n_epochs = 7
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)
    
    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []

    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]        
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
        
        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss

        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)

        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
        
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

  0%|          | 0/330 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/83 [00:00<?, ?it/s]

(Epoch 1) TRAIN LOSS:0.4001 ACC:0.84 F1:0.77 REC:0.74 PRE:0.83 LR:0.00000300


VALID LOSS:0.2414 ACC:0.91 F1:0.87 REC:0.85 PRE:0.90: 100%|██████████| 83/83 [00:29<00:00,  2.84it/s]
  0%|          | 0/330 [00:00<?, ?it/s]

(Epoch 1) VALID LOSS:0.2414 ACC:0.91 F1:0.87 REC:0.85 PRE:0.90


  0%|          | 0/83 [00:00<?, ?it/s]

...

(Epoch 6) VALID LOSS:0.2518 ACC:0.93 F1:0.90 REC:0.88 PRE:0.93


  0%|          | 0/83 [00:00<?, ?it/s]

(Epoch 7) TRAIN LOSS:0.0435 ACC:0.99 F1:0.98 REC:0.98 PRE:0.99 LR:0.00000300


VALID LOSS:0.2828 ACC:0.93 F1:0.90 REC:0.88 PRE:0.93: 100%|██████████| 83/83 [00:28<00:00,  2.88it/s]

(Epoch 7) VALID LOSS:0.2828 ACC:0.93 F1:0.90 REC:0.88 PRE:0.93


In [56]:
# torch.save(model,"drive/MyDrive/Colab Notebooks/model11NLP.pt")

In [57]:
myNLPmodel10 = torch.load("drive/MyDrive/Colab Notebooks/model11NLP.pt")

In [66]:
text = 'FREN TURUN CUY'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(myNLPmodel10.device)

logits = myNLPmodel10(subwords)[0]

label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
print("HIYA2",label)
print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

HIYA2 2
Text: FREN TURUN CUY | Label : negative (98.416%)


In [67]:
text = 'FREN GOGO NAIK TERUS'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(myNLPmodel10.device)

logits = myNLPmodel10(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
print("HIYA2",label)
print(f'Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

HIYA2 0
Label : positive (79.464%)


In [70]:
text = 'BUDI KE STOCKBIT MEMANTAU SAHAM'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(myNLPmodel10.device)

logits = myNLPmodel10(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
print("HIYA2",label)
print(f'Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

HIYA2 1
Label : neutral (99.867%)


In [60]:
%ls ./drive/MyDrive/Library/CS-3k/dataset_saham_icimtech.xlsx

./drive/MyDrive/Library/CS-3k/dataset_saham_icimtech.xlsx


In [61]:
testing_manual = pd.read_excel('./drive/MyDrive/Library/CS-3k/dataset_saham_icimtech.xlsx')

testing_manual['manual'] = testing_manual['manual'].astype(int)
testing_manual


selected_testing = testing_manual.filter(['content_original','created','likes','manual'], axis=1)
selected_testing

,content_original,created,likes,manual
0,tlkm mampir ndar 3300 sunglasses,2021-03-05 10:47:44,0,-1
1,alamkaton LKH boncos di BUMI saya malah cuan d...,2021-03-05 10:18:12,1,1
2,IHSG ANTM bentar lagi tumbang yang lain janga...,2021-03-05 10:04:09,4,0
3,IHSG maskudnya bebas pajak dividen tuh apa yah...,2021-03-05 09:32:26,0,0
4,TLKM wow HiT tlkm slightfrown mantul 3350,2021-03-05 09:15:28,0,1
...,...,...,...,...
1096,BABPW4 lumayan cuan 11 hari ini rofl yg takut...,2021-03-01 15:52:26,2,0
1097,PNBS otw ke 2000 bro sis BRIS BANK BBRI ANTM,2021-03-01 15:27:36,2,0
1098,BRIS apa yg terjadi dgn anda sayangku apakah a...,2021-03-01 15:27:35,0,1
1099,BRIS,2021-03-01 15:24:37,1,0


In [62]:
import re, string, timeit

s = "string. With. Punctuation"
s.translate(str.maketrans('', '', string.punctuation))
s

'string. With. Punctuation'

In [79]:
def check_sent(comments):
  # comments = 'FREN Jika harga lewat 78, HATI-HATI !!!'
  # subwords = tokenizer.encode(comments)
  subwords = tokenizer.encode(comments, truncation=True, max_length=512)
  subwords = torch.LongTensor(subwords).view(1, -1).to(myNLPmodel10.device)

  logits = myNLPmodel10(subwords)[0]
  label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

  if label == 1 : 
    return 0
  elif label == 2 :
    return -1
  else :
    return 1
# ORI
  # if label == 1 : 
  #   return 'Netral'
  # elif label == 2 :
  #   return 'Negatif'
  # else :
  #   return "Positif"

selected_testing['label_by_model'] = selected_testing['content_original'].apply(check_sent)
selected_testing

,content_original,created,likes,manual,label_by_model
0,tlkm mampir ndar 3300 sunglasses,2021-03-05 10:47:44,0,-1,0
1,alamkaton LKH boncos di BUMI saya malah cuan d...,2021-03-05 10:18:12,1,1,-1
2,IHSG ANTM bentar lagi tumbang yang lain janga...,2021-03-05 10:04:09,4,0,1
3,IHSG maskudnya bebas pajak dividen tuh apa yah...,2021-03-05 09:32:26,0,0,0
4,TLKM wow HiT tlkm slightfrown mantul 3350,2021-03-05 09:15:28,0,1,0
...,...,...,...,...,...
1096,BABPW4 lumayan cuan 11 hari ini rofl yg takut...,2021-03-01 15:52:26,2,0,1
1097,PNBS otw ke 2000 bro sis BRIS BANK BBRI ANTM,2021-03-01 15:27:36,2,0,0
1098,BRIS apa yg terjadi dgn anda sayangku apakah a...,2021-03-01 15:27:35,0,1,1
1099,BRIS,2021-03-01 15:24:37,1,0,-1


In [106]:
selected_testing

,content_original,created,likes,manual,label_by_model
0,tlkm mampir ndar 3300 sunglasses,2021-03-05 10:47:44,0,-1,0
1,alamkaton LKH boncos di BUMI saya malah cuan d...,2021-03-05 10:18:12,1,1,-1
2,IHSG ANTM bentar lagi tumbang yang lain janga...,2021-03-05 10:04:09,4,0,1
3,IHSG maskudnya bebas pajak dividen tuh apa yah...,2021-03-05 09:32:26,0,0,0
4,TLKM wow HiT tlkm slightfrown mantul 3350,2021-03-05 09:15:28,0,1,0
...,...,...,...,...,...
1096,BABPW4 lumayan cuan 11 hari ini rofl yg takut...,2021-03-01 15:52:26,2,0,1
1097,PNBS otw ke 2000 bro sis BRIS BANK BBRI ANTM,2021-03-01 15:27:36,2,0,0
1098,BRIS apa yg terjadi dgn anda sayangku apakah a...,2021-03-01 15:27:35,0,1,1
1099,BRIS,2021-03-01 15:24:37,1,0,-1


In [103]:
newArr = []
dataBenar = 0;
dataSalah = 0;
total = 0;
for index, row2 in selected_testing.iterrows():
    # print(row2['manual'])
    # print(row2['label_by_model'])
    if total < 1100:
       total += 1;
    if row2['label_by_model'] != row2['manual']:
      dataSalah += 1;
    else :
      dataBenar += 1;

print(dataBenar/total)

0.6690909090909091


In [107]:
selected_testing

,content_original,created,likes,manual,label_by_model
0,tlkm mampir ndar 3300 sunglasses,2021-03-05 10:47:44,0,-1,0
1,alamkaton LKH boncos di BUMI saya malah cuan d...,2021-03-05 10:18:12,1,1,-1
2,IHSG ANTM bentar lagi tumbang yang lain janga...,2021-03-05 10:04:09,4,0,1
3,IHSG maskudnya bebas pajak dividen tuh apa yah...,2021-03-05 09:32:26,0,0,0
4,TLKM wow HiT tlkm slightfrown mantul 3350,2021-03-05 09:15:28,0,1,0
...,...,...,...,...,...
1096,BABPW4 lumayan cuan 11 hari ini rofl yg takut...,2021-03-01 15:52:26,2,0,1
1097,PNBS otw ke 2000 bro sis BRIS BANK BBRI ANTM,2021-03-01 15:27:36,2,0,0
1098,BRIS apa yg terjadi dgn anda sayangku apakah a...,2021-03-01 15:27:35,0,1,1
1099,BRIS,2021-03-01 15:24:37,1,0,-1


In [108]:
from google.colab import files

selected_testing.to_excel('dataset_saham_icimtech.xlsx')
files.download('dataset_saham_icimtech.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>